In [1]:
%matplotlib inline

import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

def play_sound(x, amp=0.5):

    p = pyaudio.PyAudio()
    # for paFloat32 sample values must be in range [-1.0, 1.0]
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=fs,
                    output=True)

    # play. May repeat with different volume values (if done interactively) 
    stream.write(amp*x)
    stream.stop_stream()
    stream.close()
    p.terminate()
    pass


def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx


def make_sound(f, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    x = (np.sin(2*np.pi*np.arange(fs*duration)*f/fs)).astype(np.float32)
    return x


def make_AM_sound(f, amp_modulator, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    x = make_sound(f, fs, duration)
    x = np.multiply(x, amp_modulator)
    return x


def make_FM_sound(f, freq_modulator, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    fs = float(fs)
    x = (np.sin(2*np.pi*np.arange(fs*duration)*np.multiply(f/fs, freq_modulator))).astype(np.float32)
    return x


def complex_tone(freqs, amps, duration):
   # make tone
    tones = list()
    for amp, f in zip(amps, freqs):
        tones.append(make_sound(f, fs=fs, duration=duration)*amp)
    tone = np.array(tones).sum(0)  # sum the frequencies together
    return tone


def power_to_db(power, conversion_val=10):
    '''
    This is the conversion from power -> dB, as taken from a stack exchange forum.
    '''
    return conversion_val*np.log10(power)


def db_to_power(db, conversion_val=10):
    '''
    This is the conversion from dB -> power, as also taken from a stack exchange forum.
    '''
    return 10 ** (db/conversion_val)


def get_fft_half(tone, fs):
    power = np.abs(np.fft.fft(tone))
    n = len(power)
    freq = np.fft.fftfreq(n, 1/float(fs))

    # just get first half of spectrum
    half_n = int(np.ceil(n/2.0))
#     power = (2.0 / n) * power[:half_n]
    power = power[:half_n]
    freq_half = freq[:half_n]
    return power, freq_half


def get_db(tone, fs):

    N = 8192  # not sure where this comes from ... :/
    win = np.hamming(N)                                                       
    x = tone[0:N] * win                             # Take a slice and multiply by a window

    sp = np.fft.rfft(x)                               # Calculate real FFT (this just means the first half)

    mag = np.abs(sp) 
    ref = np.sum(win) / 2                             # Reference : window sum and factor of 2
                                                      # because we are using half of FFT spectrum

    s_dbfs = 20 * np.log10(mag / ref)                 # Convert to dBFS
    
    freq = np.fft.fftfreq(N, 1/float(fs))
    half_n = int(np.ceil(N/2.0))  # not sure about N
    freq_half = freq[:half_n+1]
    
    return s_dbfs, freq_half


def plot_wave_and_fft(tone, fs, t_max, freq_max, power_max):
    # plot waveform
    f, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(tone)
    axs[0].set_xlim([0, t_max])
    axs[0].set_xlabel('Time (ms)')
    axs[0].set_ylabel('Amplitude')

    # get fft
    power, freq_half = get_fft_half(tone, fs)

    # plot fft
    axs[1].plot(freq_half, power, lw=4, c='k')
    axs[1].set_xlim([0, freq_max])
    axs[1].set_ylim([0, power_max])
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Power')
    return f


def plot_wave_and_db(tone, fs, t_max, freq_max):
    # plot waveform
    f, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(tone)
    axs[0].set_xlim([0, t_max])
    axs[0].set_xlabel('Time (ms)')
    axs[0].set_ylabel('Amplitude')

    # get fft
    db, freq_half = get_db(tone, fs)

    # plot fft
    axs[1].plot(freq_half, power_to_db(power), lw=4, c='k')
    axs[1].set_xlim([0, freq_max])
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('dB')
    return f



__Now, let's make some Shepard tones!!__

In [2]:
import time

from scipy.io.wavfile import write
base_dir = '/Users/ellieabrams/Desktop/Projects/Shepard/exp/tones_final'
base_dir = '/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_70db'

# params
fs=44100  # sample freq
duration=0.8
scale_idx = [0, 2, 4, 6, 7, 9, 11]
max_db = -0
min_db = -38.5
tone_length = 0.3  # desired length in seconds
notes = ['Eb', 'A', 'C']
conditions = ['shepard', 'pure', 'partial']

# semitones for each scale
#d_semitones = np.array([294, 277, 262, 247, 233, 220, 208, 196, 185, 175, 165, 156])
eb_semitones = np.array([312, 330, 350, 370, 392, 415, 440, 466, 494, 523, 554, 587])[::-1]
c_semitones = np.array([494, 466, 440, 415, 392, 370, 349, 330, 311, 294, 277, 262])
a_semitones = np.array([415, 392, 370, 349, 330, 311, 294, 277, 262, 247, 233, 220])

for note in notes:

    for cond in conditions:

        if note == 'Eb':
            semitones = eb_semitones
        elif note == 'C':
            semitones = c_semitones
        elif note == 'A':
            semitones = a_semitones
        
        if cond == 'shepard':
            n_partials = 6
            linear_db = np.linspace(min_db, max_db, len(semitones))[::-1]
            non_linear_amp = np.sqrt(db_to_power(linear_db))
            partial_scaling = non_linear_amp / 15
            min_amp = np.min(partial_scaling)
            max_amp = np.max(partial_scaling)

        elif cond == 'pure':
            n_partials = 1
            semitones = np.array(semitones.tolist() + [semitones[-1]*2])
            linear_db = np.ones_like(semitones)
            non_linear_amp = np.sqrt(db_to_power(linear_db))
            partial_scaling = non_linear_amp / 15
            min_amp = np.min(partial_scaling)
            max_amp = np.max(partial_scaling)            

        elif cond == 'partial':
            n_partials = 6
            semitones = np.array(semitones.tolist() + [semitones[-1]*2])
            linear_db = np.ones_like(semitones)
            non_linear_amp = np.sqrt(db_to_power(linear_db))
            partial_scaling = non_linear_amp / 15
            min_amp = np.min(partial_scaling)
            max_amp = np.max(partial_scaling)       
            
        # bin for tones
        tones = list()

        # loop through each note of the a scale
        for ii, f0 in enumerate(semitones):
            print f0
            # start the timer
            t0 = time.time()

            # get the freqs
            freqs = [f0*p for p in range(1, n_partials+1)]
            if cond == 'partial':  # remove f0 for partial
                freqs = freqs[1:]
            print f0
            # scale the odd partials by scalar
            attenuation_1_3 = partial_scaling[ii]  # this is the intermediary value (neither min nor max -- somewhere on the linear line)
            attenuation_6 = partial_scaling[::-1][ii]  # this is the intermediary value for the last partial

            amps = list()
            for jj in range(1, n_partials+1):  # loop through partial numbers (1 - 6, not 0-5)

                # if it is partial 5 -- stay at the min
                if jj == 5:
                    amps.append(min_amp)  # stays low constantly

                # if it is partial 6 -- give attenuation up
                elif jj == 6:
                    amps.append(attenuation_6) # scales in the opposite direction of 1&3     

                # if it is partial 2 or 4 -- keep at max
                elif jj == 2 or jj == 4:
                    amps.append(max_amp)  # if it's even, do not mess with the amp (it stays the same throughout)

               # if it is partial 1 or 3 -- give attenuation down
                elif jj == 1 or jj == 3:
                    amps.append(attenuation_1_3)  # if it's odd, attenuate.

                else:
                    raise ValueError('You screwed up somewhere.')

            # make tone
            tone = complex_tone(freqs, amps, duration)
            tones.append(tone)

#             if ii in scale_idx:
#                 # listen        
#                 play_sound(tone)

#                 dbs, freq_half = get_db(tone, fs)
#                 plt.plot(freq_half, dbs)
#                 for lin_db in linear_db[scale_idx]:
#                     plt.axhline(lin_db, c='r')
#                 plt.xlim([0, 3000])
#                 plt.ylim([-50, 0])
#                 plt.title(f0)
#                 plt.show()

            fname = '%s/%s-%s_%s.wav' % (base_dir, cond, note, f0)
            write(fname, fs, tone[0:int(fs*tone_length)])
            print(fname)

587
587


IOError: [Errno 2] No such file or directory: '/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_70db/shepard-Eb_587.wav'

In [12]:
c_semitones[scale_idx]

array([494, 440, 392, 349, 330, 294, 262])

In [5]:
amps

[0.25, 0.25, 0.25, 0.25, 0.25, 0.25]

In [6]:
expected_trial_n = (12 * 3) + (13 * 3) + (13 * 3)
print('we should have %s files total.' % (expected_trial_n))

we should have 114 files total.
